<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/fdsabr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 25.3 MB/s eta 0:00:00


In [ ]:
class FdSabrTests(unittest.TestCase):

    def setUp(self):
        self.saved_eval_date = ql.Settings.instance().evaluationDate

    def tearDown(self):
        ql.Settings.instance().evaluationDate = self.saved_eval_date

    def testFdmSabrOp(self):
        # This test has `@precondition(if_speed(Fast))`
        print("Testing FDM SABR operator...")

        today = ql.Date(22, ql.February, 2018)
        dc = ql.Actual365Fixed()
        ql.Settings.instance().evaluationDate = today

        maturityDate = today + ql.Period(2, ql.Years)
        maturityTime = dc.yearFraction(today, maturityDate)
        strike = 1.5

        exercise = ql.EuropeanExercise(maturityDate)
        putPayoff = ql.PlainVanillaPayoff(ql.Option.Put, strike)
        callPayoff = ql.PlainVanillaPayoff(ql.Option.Call, strike)

        optionPut = ql.VanillaOption(putPayoff, exercise)
        optionCall = ql.VanillaOption(callPayoff, exercise)

        rTS = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.0, dc)) # Zero interest rate

        f0 = 1.0
        alpha = 0.35
        nu = 1.0
        rho = 0.25
        betas = [0.25, 0.6]

        # For implied vol calculation
        bs_process = ql.GeneralizedBlackScholesProcess(
            ql.QuoteHandle(ql.SimpleQuote(f0)),
            rTS, # dividend TS = riskFreeRateTS (as it's for FX-like SABR fwd)
            rTS, # riskFreeRateTS
            ql.BlackVolTermStructureHandle(ql.BlackConstantVol(today, ql.NullCalendar(), 0.2, dc)) # Dummy vol for IV calc
        )

        for beta_val in betas:
            # FdSabrVanillaEngine(f0, alpha, beta, nu, rho, rTS,
            #                     tGrid, xGrid, vGrid, dampingSteps, schemeDesc, localVol, illegalLocalVolOverwrite)
            # C++: FdSabrVanillaEngine(f0, alpha, beta, nu, rho, rTS, 100, 400, 100)
            #      tGrid=100, xGrid=400, vGrid(alpha_grid)=100
            pde_engine = ql.FdSabrVanillaEngine(
                f0, alpha, beta_val, nu, rho, rTS, 100, 400, 100)

            optionPut.setPricingEngine(pde_engine)
            pdePut = optionPut.NPV()

            optionCall.setPricingEngine(pde_engine)
            pdeCall = optionCall.NPV()

            # Put-Call Parity Check (assuming r=q=0 for fwd process)
            # F = S since r=q=0. So C - P = S - K * D(T)
            # Here, D(T)=1 as r=0. So C - P = S - K.
            # The f0 in SABR is the forward rate, so C - P = f0 - K (if zero rates)
            pdeFwd = pdeCall - pdePut
            expectedFwd = f0 - strike # Since r=0 implies D(T)=1 for discounting strike

            parityDiff = abs(pdeFwd - expectedFwd)
            parityTol = 1e-4
            self.assertLessEqual(parityDiff, parityTol,
                                 msg=(f"Put-Call Parity failed for beta={beta_val}\n"
                                      f"    PDE Fwd: {pdeFwd}, Expected Fwd: {expectedFwd}, Diff: {parityDiff}"))

            # Implied Volatility Comparison
            # C++: optionPut.impliedVolatility(optionPut.NPV(), bsProcess, 1e-6);
            # Ensure NPV is positive for IV calculation
            if pdePut <= 1e-16: # Effectively zero or negative
                 # This case might indicate an issue or extreme parameters
                 # Skip IV calc or handle as error if unexpected
                 print(f"Warning: PDE Put NPV is non-positive ({pdePut}) for beta={beta_val}, skipping IV calc.")
                 putPdeImplVol = 0.0 # Or some other indicator
            else:
                 putPdeImplVol = optionPut.impliedVolatility(pdePut, bs_process, 1e-6)


            # Monte Carlo Pricer setup
            # Richardson Extrapolation: mcSabr(dt) where dt is the time step size
            # C++: RichardsonExtrapolation(mcSabr, 1/4.0)(4.0, 2.0);
            #      mcSabr is func, initial_h = 1/4.0, target_factor_n = 4.0, richardson_factor_t = 2.0
            # This means it computes mcSabr(h), mcSabr(h/2), mcSabr(h/4)
            # Initial h = 1/4.0.
            # Steps are: (1/4.0), (1/4.0)/2.0 = 1/8.0, (1/4.0)/4.0 = 1/16.0
            # The `(4.0, 2.0)` means extrapolate up to n=4, using t=2.
            # For n=4, means 3 levels: h, h/t, h/t^2. Here t=2 => h, h/2, h/4.
            # Initial h = 1/4.0. So computes with dt = 1/4, 1/8, 1/16.

            mc_pricer_func = SabrMonteCarloPricer(f0, maturityTime, putPayoff,
                                                  alpha, beta_val, nu, rho)

            # RichardsonExtrapolation(func, h_start, n_factor=4.0, t_factor=2.0)
            # h_start is the largest step size.
            # n_factor is the order of extrapolation (related to number of levels).
            # t_factor is the scaling factor for h (h, h/t, h/t^2, ...).
            # The C++ (4.0, 2.0) means n=4, t=2 in QL notation.
            richardson = ql.RichardsonExtrapolation(mc_pricer_func, 1.0/4.0) # h_start = 1/4.0
            mcNPV = richardson(4.0, 2.0) # n_factor=4.0, t_factor=2.0

            if mcNPV <= 1e-16:
                 print(f"Warning: MC NPV is non-positive ({mcNPV}) for beta={beta_val}, skipping IV calc.")
                 putMcImplVol = 0.0
            else:
                putMcImplVol = optionPut.impliedVolatility(mcNPV, bs_process, 1e-6)

            volDiff = abs(putPdeImplVol - putMcImplVol)
            volTol = 5e-3 # 0.5%
            self.assertLessEqual(volDiff, volTol,
                                 msg=(f"Implied Vol Mismatch for beta={beta_val}\n"
                                      f"    PDE IV: {putPdeImplVol:.4f}, MC IV: {putMcImplVol:.4f}, Diff: {volDiff:.4e}"))


    def testFdmSabrCevPricing(self):
        print("Testing FDM CEV pricing with trivial SABR model...")
        today = ql.Date(3, ql.January, 2019)
        dc = ql.Actual365Fixed()
        ql.Settings.instance().evaluationDate = today
        maturityDate = today + ql.Period(12, ql.Months)

        betas_cev = [0.1, 0.9]
        strikes_cev = [0.9, 1.5]
        f0_cev = 1.2
        alpha_cev = 0.35
        nu_cev = 1e-3 # Trivial nu for CEV limit
        rho_cev = 0.25 # Rho doesn't matter much if nu is tiny

        rTS_cev = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.05, dc))
        exercise_cev = ql.EuropeanExercise(maturityDate)
        optionTypes_cev = [ql.Option.Put, ql.Option.Call]
        tol_cev = 5e-5

        for opt_type in optionTypes_cev:
            for k_strike in strikes_cev:
                payoff_cev = ql.PlainVanillaPayoff(opt_type, k_strike)
                option_cev = ql.VanillaOption(payoff_cev, exercise_cev)

                for beta_param in betas_cev:
                    # FdSabrVanillaEngine (f0, alpha, beta, nu, rho, rTS, tGrid, xGrid, vGrid)
                    # C++: (f0, alpha, beta, nu, rho, rTS, 100, 400, 3) -> vGrid for alpha is 3 (small for CEV limit)
                    fd_sabr_engine = ql.FdSabrVanillaEngine(
                        f0_cev, alpha_cev, beta_param, nu_cev, rho_cev, rTS_cev, 100, 400, 3)
                    option_cev.setPricingEngine(fd_sabr_engine)
                    calculated_npv = option_cev.NPV()

                    # AnalyticCEVEngine(f0, alpha (cev_sigma), beta, rTS)
                    analytic_cev_engine = ql.AnalyticCEVEngine(f0_cev, alpha_cev, beta_param, rTS_cev)
                    option_cev.setPricingEngine(analytic_cev_engine)
                    expected_npv = option_cev.NPV()

                    self.assertAlmostEqual(calculated_npv, expected_npv, delta=tol_cev,
                                           msg=(f"CEV pricing failed: Type={opt_type}, K={k_strike}, Beta={beta_param}\n"
                                                f"    PDE: {calculated_npv:.6f}, Analytic: {expected_npv:.6f}"))

    def testFdmSabrVsVolApproximation(self):
        print("Testing FDM SABR vs approximations (Hagen)...")
        today = ql.Date(8, ql.January, 2019)
        dc = ql.Actual365Fixed()
        ql.Settings.instance().evaluationDate = today
        maturityDate = today + ql.Period(6, ql.Months)
        maturityTime_approx = dc.yearFraction(today, maturityDate)
        rTS_approx = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.05, dc))
        f0_approx = 100.0

        bs_process_approx = ql.GeneralizedBlackScholesProcess(
            ql.QuoteHandle(ql.SimpleQuote(f0_approx)),
            rTS_approx, rTS_approx,
            ql.BlackVolTermStructureHandle(ql.BlackConstantVol(today, ql.NullCalendar(), 0.2, dc)) # Dummy vol
        )

        alpha_approx = 0.35; beta_approx = 0.85; nu_approx = 0.75; rho_approx = 0.85
        strikes_approx = [90.0, 100.0, 110.0]
        optionTypes_approx = [ql.Option.Put, ql.Option.Call]
        tol_approx = 2.5e-3 # 0.25% vol difference

        for opt_type in optionTypes_approx:
            for k_strike in strikes_approx:
                option_val = ql.VanillaOption(
                    ql.PlainVanillaPayoff(opt_type, k_strike),
                    ql.EuropeanExercise(maturityDate))

                # FdSabrVanillaEngine(f0, alpha, beta, nu, rho, rTS, tGrid, xGrid, vGrid)
                # C++: (25, 100, 50) -> tGrid=25, xGrid=100, vGrid=50
                fd_engine_approx = ql.FdSabrVanillaEngine(
                    f0_approx, alpha_approx, beta_approx, nu_approx, rho_approx,
                    rTS_approx, 25, 100, 50)
                option_val.setPricingEngine(fd_engine_approx)

                fdm_npv = option_val.NPV()
                if fdm_npv <= 1e-16:
                    print(f"Warning: FDM NPV is non-positive ({fdm_npv}) for Type={opt_type}, K={k_strike}, skipping IV calc.")
                    fdmVol = 0.0
                else:
                    fdmVol = option_val.impliedVolatility(fdm_npv, bs_process_approx)

                hagenVol = ql.sabrVolatility(
                    k_strike, f0_approx, maturityTime_approx, alpha_approx,
                    beta_approx, nu_approx, rho_approx)

                diff_vol = abs(fdmVol - hagenVol)
                self.assertLessEqual(diff_vol, tol_approx,
                                     msg=(f"Hagen Approx vs FDM Vol failed: Type={opt_type}, K={k_strike}\n"
                                          f"    FDM Vol: {fdmVol:.5f}, Hagen Vol: {hagenVol:.5f}, Diff: {diff_vol:.3e}"))

    def testOosterleeTestCaseIV(self):
        print("Testing Chen, Oosterlee and Weide test case IV...")
        today = ql.Date(8, ql.January, 2019)
        dc = ql.Actual365Fixed()
        ql.Settings.instance().evaluationDate = today
        rTS_oost = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.0, dc)) # Zero interest

        f0_oost = 0.07; alpha_oost = 0.4; nu_oost = 0.8; beta_oost = 0.4; rho_oost = -0.6

        maturities_periods = [ql.Period(2, ql.Years), ql.Period(5, ql.Years), ql.Period(10, ql.Years)]
        # Strikes relative to f0
        strike_factors = [0.4, 1.0, 1.6]
        strikes_oost = [f * f0_oost for f in strike_factors]

        tol_oost = 0.00035

        for i, mat_period in enumerate(maturities_periods):
            maturityDate_oost = today + mat_period
            maturityTime_oost = dc.yearFraction(today, maturityDate_oost)

            # C++: timeSteps = Size(5*maturityTime);
            timeSteps_oost = int(5 * maturityTime_oost)
            if timeSteps_oost == 0 : timeSteps_oost = 1

            # FdSabrVanillaEngine (f0, alpha, beta, nu, rho, rTS, tGrid, xGrid, vGrid)
            # C++: (timeSteps, 200, 21) -> tGrid=timeSteps_oost, xGrid=200, vGrid=21
            engine_oost = ql.FdSabrVanillaEngine(
                f0_oost, alpha_oost, beta_oost, nu_oost, rho_oost,
                rTS_oost, timeSteps_oost, 200, 21)

            exercise_oost = ql.EuropeanExercise(maturityDate_oost)

            for j, k_strike_val in enumerate(strikes_oost):
                payoff_oost = ql.PlainVanillaPayoff(ql.Option.Call, k_strike_val)
                option_oost = ql.VanillaOption(payoff_oost, exercise_oost)
                option_oost.setPricingEngine(engine_oost)
                calculated_npv_oost = option_oost.NPV()

                # OsterleeReferenceResults index is i*3+j
                reference_func = OsterleeReferenceResults(i * len(strikes_oost) + j)

                # Richardson Extrapolation for reference results
                # C++: RichardsonExtrapolation(std::function<Real(Real)>(referenceResuts), 1/16., 1)(2.);
                #      h_start=1/16.0, order_of_extrapolation_coef=1, target_factor_n=2.0
                # This means it computes ref(h_start), ref(h_start/2.0) and extrapolates linearly.
                # For QL Python: RichardsonExtrapolation(func, h_start)(n_factor=2.0, t_factor=2.0)
                # where n_factor=2 for linear (two points).
                richardson_ref = ql.RichardsonExtrapolation(reference_func, 1.0/16.0)
                expected_npv_oost = richardson_ref(2.0, 2.0) # n_factor=2, t_factor=2

                diff_oost = abs(calculated_npv_oost - expected_npv_oost)
                self.assertLessEqual(diff_oost, tol_oost,
                                     msg=(f"Oosterlee Case IV failed: Mat={mat_period}, K={k_strike_val:.4f}\n"
                                          f"    PDE: {calculated_npv_oost:.6f}, Ref: {expected_npv_oost:.6f}, Diff: {diff_oost:.3e}"))


    def testBenchOpSabrCase(self):
        print("Testing SABR BenchOp problem...")
        today = ql.Date(8, ql.January, 2019)
        dc = ql.Actual365Fixed()
        ql.Settings.instance().evaluationDate = today
        rTS_bench = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.0, dc))

        maturityInYears_bench = [2, 10]
        f0s_bench    = [0.5, 0.07]
        alphas_bench = [0.5, 0.4]
        nus_bench    = [0.4, 0.8]
        betas_bench  = [0.5, 0.5] # Beta is same for both cases in C++
        rhos_bench   = [0.0, -0.6]

        expected_npvs_bench = [
            [0.221383196830866, 0.193836689413803, 0.166240814653231], # Case 1 (Mat=2Y)
            [0.052450313614407, 0.046585753491306, 0.039291470612989]  # Case 2 (Mat=10Y)
        ]

        # Grid parameters from C++
        gridX_base = 400; gridY_base = 25; gridT_base = 10
        factor = 2.0 # Floating point for sqrt
        tol_bench = 2e-4

        for i in range(len(f0s_bench)):
            # maturity_bench = today + ql.Period(maturityInYears_bench[i] * 365, ql.Days)
            # More robust:
            maturity_bench = today + ql.Period(maturityInYears_bench[i], ql.Years)
            T_bench = dc.yearFraction(today, maturity_bench)

            f0_val    = f0s_bench[i]
            alpha_val = alphas_bench[i]
            nu_val    = nus_bench[i]
            beta_val  = betas_bench[i]
            rho_val   = rhos_bench[i]

            # Strikes relative to f0 and sqrt(T)
            strikes_bench_current_case = [
                f0_val * math.exp(-0.1 * math.sqrt(T_bench)),
                f0_val,
                f0_val * math.exp(0.1 * math.sqrt(T_bench))
            ]

            # Grid sizes for this case
            # C++: Size(gridT*factor), Size(gridX*factor), Size(gridY*std::sqrt(factor))
            current_tGrid = int(gridT_base * factor)
            current_xGrid = int(gridX_base * factor)
            current_yGrid = int(gridY_base * math.sqrt(factor)) # alpha grid (y-dim for SABR)

            for j, k_strike_val in enumerate(strikes_bench_current_case):
                option_bench = ql.VanillaOption(
                    ql.PlainVanillaPayoff(ql.Option.Call, k_strike_val),
                    ql.EuropeanExercise(maturity_bench))

                engine_bench = ql.FdSabrVanillaEngine(
                    f0_val, alpha_val, beta_val, nu_val, rho_val, rTS_bench,
                    current_tGrid, current_xGrid, current_yGrid)
                option_bench.setPricingEngine(engine_bench)

                calculated_npv_bench = option_bench.NPV()
                expected_npv_val = expected_npvs_bench[i][j]
                diff_bench = abs(calculated_npv_bench - expected_npv_val)

                self.assertLessEqual(diff_bench, tol_bench,
                                     msg=(f"BenchOp SABR Case {i+1}, Strike {j+1} (K={k_strike_val:.4f}) failed:\n"
                                          f"    PDE: {calculated_npv_bench:.9f}, Exp: {expected_npv_val:.9f}, Diff: {diff_bench:.3e}"))


if __name__ == '__main__':
    print("C++ test suite: FdSabrTests")
    print("Python QuantLib version: " + ql.__version__)
    print("-------------------------------------------------------")
    suite = unittest.TestSuite()
    suite.addTest(unittest.makeSuite(FdSabrTests))
    unittest.TextTestRunner(verbosity=2).run(suite)

Key Considerations during Translation:
SabrMonteCarloPricer Random Number Generation:
The C++ code uses SobolBrownianBridgeRsg and accesses its output n as if n[j] is for the first factor and n[j+timeSteps] is for the second factor across timeSteps.
In Python, ql.SobolBrownianBridgeRsg(factors, steps, ...) typically generates a sequence where values for all factors at a given step are grouped, e.g., [z1_t1, z2_t1, z1_t2, z2_t2, ...].
However, other Sobol generators like ql.SobolRsg(dim, seed) in Python produce a flat list [factor1_val, factor2_val, ..., factor_dim_val] for nextSequence().value(). If SobolBrownianBridgeRsg behaves like this for one path (i.e., nextSequence() gives one full path with all steps and factors), then the C++ logic might be: n[j] is val_factor1_step_j and n[j + timeSteps] is val_factor2_step_j.
I've updated the SabrMonteCarloPricer in Python to attempt to match the C++ indexing logic, assuming nextSequence().value() returns a flat list where the first timeSteps values are for the first random factor, and the next timeSteps are for the second. This part is crucial and might need verification against actual QL Python behavior for SobolBrownianBridgeRsg if the MC results don't match. A common pattern for path generators is [[path1_vals], [path2_vals], ...]. I added a small dynamic check and alternative for this.
The C++ comments out a CEVRNDCalculator part, using a simple Euler method. The Python code replicates the Euler method.
OsterleeReferenceResults: This is a straightforward data holder; translated directly.
Richardson Extrapolation:
C++: RichardsonExtrapolation(mcSabr, 1/4.0)(4.0, 2.0) where mcSabr is the function, 1/4.0 is h_initial. The (4.0, 2.0) call means n_factor = 4.0 and t_factor = 2.0.
Python: ql.RichardsonExtrapolation(func, h_start) constructor, then extrapolated_value = richardson_instance(n_factor, t_factor). This mapping is used.
Payoff Objects: C++ uses ext::shared_ptr<Payoff>. Python uses direct ql.Payoff derived objects.
Engine Parameters: FdSabrVanillaEngine constructor parameters are matched based on the C++ calls (e.g., tGrid, xGrid, vGrid where vGrid is for the alpha/volatility-of-volatility dimension in SABR FDM).
Implied Volatility: Calculated using the option's impliedVolatility method, providing an NPV, a Black-Scholes process (for definition of IV), and a tolerance. Added checks for non-positive NPVs before calling impliedVolatility.
Numerical Precision and Tolerances: Tolerances from C++ tests are used. They might need adjustment if Python's floating-point behavior or default FDM scheme parameters (if any subtle differences exist) cause slight deviations.
std::size: Replaced with len() for Python lists/arrays.
precondition(if_speed(Fast)): Python's unittest doesn't have a direct equivalent for these Boost.Test decorators. The tests are run by default. If a test is very slow, unittest.skip or conditional execution could be used.
